In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hypothyroidism"
cohort = "GSE75678"

# Input paths
in_trait_dir = "../../input/GEO/Hypothyroidism"
in_cohort_dir = "../../input/GEO/Hypothyroidism/GSE75678"

# Output paths
out_data_file = "../../output/preprocess/Hypothyroidism/GSE75678.csv"
out_gene_data_file = "../../output/preprocess/Hypothyroidism/gene_data/GSE75678.csv"
out_clinical_data_file = "../../output/preprocess/Hypothyroidism/clinical_data/GSE75678.csv"
json_path = "../../output/preprocess/Hypothyroidism/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data
# from breast cancer tumors. It's not miRNA or methylation data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For Hypothyroidism trait
# Looking at the sample characteristics, we find personal pathological history at index 21,
# which includes "Hypothyroidism" as one of the values
trait_row = 21

# For age
# Age at diagnosis is available at index 19
age_row = 19

# For gender
# Gender is available at index 1, but it shows only "Female" for all samples
# Since it's constant (all Female), we'll set it to None as it's not useful for association studies
gender_row = None

# 2.2 Data Type Conversion
# Convert trait function - binary variable (0 = No hypothyroidism, 1 = Has hypothyroidism)
def convert_trait(value):
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'Hypothyroidism' in value:
        return 1
    else:
        return 0

# Convert age function - continuous variable
def convert_age(value):
    if pd.isna(value):
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    try:
        return float(value)
    except:
        return None

# Convert gender function - not needed as gender data is constant (all Female)
def convert_gender(value):
    return None

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# If trait data is available, extract clinical features
if trait_row is not None:
    # Create a more appropriate structure for the clinical data
    # First, identify all unique sample IDs
    sample_characteristics = {
        0: ['tissue: Tumor Sample of Breast Cancer'], 
        1: ['gender: Female'], 
        2: ['rna ng/ul: 1083', 'rna ng/ul: 343', 'rna ng/ul: 111', 'rna ng/ul: 307', 'rna ng/ul: 401', 'rna ng/ul: 475', 'rna ng/ul: 728', 'rna ng/ul: 143.6', 'rna ng/ul: 224.7', 'rna ng/ul: 1458.3', 'rna ng/ul: 164', 'rna ng/ul: 370.2', 'rna ng/ul: 419.5', 'rna ng/ul: 693.6', 'rna ng/ul: 291.4', 'rna ng/ul: 1566.4', 'rna ng/ul: 69', 'rna ng/ul: 625.4', 'rna ng/ul: 151.6', 'rna ng/ul: 127.7', 'rna ng/ul: 1116.8', 'rna ng/ul: 333.9', 'rna ng/ul: 182.1', 'rna ng/ul: 437.4', 'rna ng/ul: 439', 'rna ng/ul: 178.2', 'rna ng/ul: 1365', 'rna ng/ul: 670', 'rna ng/ul: 840.6', 'rna ng/ul: 725'],
        19: ['age at diagnosis: 45', 'age at diagnosis: 41', 'age at diagnosis: 59', 'age at diagnosis: 57', 'age at diagnosis: 42', 'age at diagnosis: 49', 'age at diagnosis: 54', 'age at diagnosis: 31', 'age at diagnosis: 70', 'age at diagnosis: 44', 'age at diagnosis: 50', 'age at diagnosis: 56', 'age at diagnosis: 51', 'age at diagnosis: 58', 'age at diagnosis: 55', 'age at diagnosis: 71', 'age at diagnosis: 40', 'age at diagnosis: 62', 'age at diagnosis: 87', 'age at diagnosis: 36', 'age at diagnosis: 43', 'age at diagnosis: 48', 'age at diagnosis: 66', 'age at diagnosis: 53', 'age at diagnosis: 35', 'age at diagnosis: 68', 'age at diagnosis: 46'],
        21: ['personal pathological hystory: Neg', 'personal pathological hystory: Rheumatoid Arthritis', 'personal pathological hystory: Hypertension', 'personal pathological hystory: Apendicitis', 'personal pathological hystory: Hypertension and Diabetes', 'personal pathological hystory: Hypothyroidism', 'personal pathological hystory: Diabetes', 'personal pathological hystory: Ocular Surgery', 'personal pathological hystory: 3 C sections', 'personal pathological hystory: 0', 'personal pathological hystory: C section', 'personal pathological hystory: Hysterechtomy', 'personal pathological hystory: Dyslipidemia', 'personal pathological hystory: Hypertension and Rheumatoid Artritis', 'personal pathological hystory: Knee Surgery', 'personal pathological hystory: Venous insuficiency']
    }
    
    # Determine the maximum number of samples based on the traits we're interested in
    max_samples = max([len(sample_characteristics[row]) for row in [trait_row, age_row] if row is not None])
    
    # Create a DataFrame with one sample per row
    samples = []
    for i in range(max_samples):
        sample_data = {}
        
        # Add trait data if available for this sample
        if trait_row is not None and i < len(sample_characteristics[trait_row]):
            sample_data['characteristics'] = sample_characteristics[trait_row][i]
            sample_data['row'] = trait_row
            samples.append(sample_data.copy())
        
        # Add age data if available for this sample
        if age_row is not None and i < len(sample_characteristics[age_row]):
            sample_data['characteristics'] = sample_characteristics[age_row][i] 
            sample_data['row'] = age_row
            samples.append(sample_data.copy())
    
    # Convert to DataFrame
    clinical_data = pd.DataFrame(samples)
    
    # Extract clinical features if there is data
    if not clinical_data.empty:
        # Use the geo_select_clinical_features function to extract features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        print(preview_df(clinical_features))
        
        # Save the clinical features to CSV
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=False)
    else:
        print("No clinical data available for processing.")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Load necessary libraries
import os
import json
import pandas as pd
import numpy as np
import gzip
from typing import Callable, Dict, Any, Optional

# First, list files in the cohort directory to see what's available
print(f"Files in {in_cohort_dir}:")
files_in_dir = os.listdir(in_cohort_dir)
print(files_in_dir)

# Extract sample characteristics from series matrix file
sample_characteristics = None
series_matrix_file = None

# Find the series matrix file
for file in files_in_dir:
    if 'series_matrix.txt' in file:
        series_matrix_file = os.path.join(in_cohort_dir, file)
        break

if series_matrix_file:
    print(f"Found series matrix file: {series_matrix_file}")
    try:
        # Open the gzipped file and extract relevant sections
        characteristics_lines = []
        with gzip.open(series_matrix_file, 'rt') as f:
            in_characteristics = False
            for line in f:
                if line.startswith('!Sample_characteristics_ch1'):
                    characteristics_lines.append(line.strip())
                elif line.startswith('!Sample_geo_accession'):
                    # Get sample IDs
                    sample_ids = line.strip().split('\t')[1:]
                # Stop when we reach the data section
                elif line.startswith('!series_matrix_table_begin'):
                    break
        
        if characteristics_lines:
            # Process the characteristics lines into a DataFrame
            data = []
            for i, line in enumerate(characteristics_lines):
                row_values = line.split('\t')[1:]  # Skip the first element (header)
                data.append(row_values)
            
            # Create a DataFrame with the characteristics data
            if data:
                sample_characteristics = pd.DataFrame(data)
                sample_characteristics.insert(0, '!Sample_geo_accession', [f"Row_{i}" for i in range(len(data))])
                
                # Display some information about the characteristics
                print(f"Found {len(characteristics_lines)} characteristic rows")
                
                # Get unique values for each row to analyze content
                unique_values = {}
                for i in range(len(sample_characteristics)):
                    if i < sample_characteristics.shape[0]:
                        values = [val.strip() if isinstance(val, str) else val for val in sample_characteristics.iloc[i, 1:]]
                        unique_values[i] = list(set([val for val in values if pd.notna(val)]))
                
                print(f"Unique values in sample characteristics:")
                for key, values in unique_values.items():
                    print(f"Row {key}: {values}")
    except Exception as e:
        print(f"Error processing series matrix file: {e}")
else:
    print("Series matrix file not found")

# Check for gene expression data availability
is_gene_available = series_matrix_file is not None
print(f"Gene expression data available: {is_gene_available}")

# Analyze unique values to determine trait, age, and gender rows
trait_row = None
age_row = None
gender_row = None

# Set these based on the analysis of unique values
if sample_characteristics is not None:
    # Search for hypothyroidism-related information
    for i in range(len(sample_characteristics)):
        row_values = [str(val).lower() for val in sample_characteristics.iloc[i, 1:] if not pd.isna(val)]
        row_text = ' '.join(row_values)
        
        # Check for trait information
        if any(term in row_text for term in ['hypothyroid', 'thyroid', 'diagnosis', 'disease', 'condition', 'status', 'patient']):
            trait_row = i
            print(f"Found trait information in row {i}: {unique_values[i]}")
        
        # Check for age information
        elif any(term in row_text for term in ['age', 'year old']):
            age_row = i
            print(f"Found age information in row {i}: {unique_values[i]}")
        
        # Check for gender information
        elif any(term in row_text for term in ['gender', 'sex', 'male', 'female']):
            gender_row = i
            print(f"Found gender information in row {i}: {unique_values[i]}")

# Define conversion functions
def convert_trait(value: str) -> int:
    """Convert hypothyroidism status to binary format (1=patient, 0=control)"""
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'hypothyroid' in value or 'patient' in value or 'case' in value or 'disease' in value:
        return 1
    elif 'control' in value or 'normal' in value or 'healthy' in value:
        return 0
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age to continuous numerical value"""
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Check for common age patterns
        import re
        age_match = re.search(r'(\d+(\.\d+)?)', value)
        if age_match:
            return float(age_match.group(1))
        else:
            return None
    except:
        return None

def convert_gender(value: str) -> int:
    """Convert gender to binary format (0=female, 1=male)"""
    if pd.isna(value):
        return None
    
    value = str(value).lower()
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value in ['female', 'f', 'woman', 'women']:
        return 0
    elif value in ['male', 'm', 'man', 'men']:
        return 1
    else:
        return None

# Determine if trait data is available
is_trait_available = trait_row is not None
print(f"Trait data available: {is_trait_available}")

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Clinical Feature Extraction (if trait data and sample characteristics are available)
if trait_row is not None and sample_characteristics is not None:
    # Extract clinical features using the helper function
    clinical_features = geo_select_clinical_features(
        clinical_df=sample_characteristics,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")
else:
    print("Clinical feature extraction skipped: trait data or sample characteristics not available")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


### Step 5: Gene Identifier Review

In [ ]:
# These identifiers appear to be just sequential numbers (1, 2, 3, etc.) which are not
# human gene symbols. Human gene symbols typically look like BRCA1, TP53, etc.
# These numeric IDs will need to be mapped to proper gene symbols.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify which columns in the gene annotation data correspond to gene IDs and gene symbols
# From the preview, we can see the gene expression data uses numeric IDs (1, 2, 3, etc.)
# The gene annotation data has an 'ID' column that appears to match these, and a 'GENE_SYMBOL' column for gene symbols

# First, let's check how many rows have gene symbols
num_rows_with_gene_symbols = gene_annotation['GENE_SYMBOL'].count()
print(f"Number of rows with gene symbols: {num_rows_with_gene_symbols} out of {len(gene_annotation)}")

# 2. Extract the mapping between gene IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'GENE_SYMBOL')
print(f"Generated gene mapping with {len(gene_mapping)} rows")
print("Preview of gene mapping:")
print(preview_df(gene_mapping))

# 3. Apply the gene mapping to convert from probe-level to gene-level expression
print("\nConverting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Generated gene expression data with {len(gene_data)} genes")
print("Preview of first few genes:")
print(gene_data.index[:20])

# Save the gene expression data
if not gene_data.empty:
    # Ensure the output directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
else:
    print("Warning: Gene expression data is empty, no file saved")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols and extract from step 3 and 6
# Load the gene expression data (already loaded from Step 6)
print(f"Gene data shape before normalization: {gene_data.shape}")

try:
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    
    if normalized_gene_data.empty:
        print("Normalization resulted in empty dataframe. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data to the output file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene data: {e}. Using original gene data instead.")
    normalized_gene_data = gene_data
    # Save the original gene data if normalization fails
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)

# 2. Link clinical and genetic data
# Use the trait_row identified in Step 2 (trait_row = 1) to extract trait data
is_trait_available = trait_row is not None

if is_trait_available:
    # Extract clinical features using the function and conversion methods from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Save clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
else:
    # Create a minimal dataframe with just the trait column
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No trait data available, creating minimal dataframe for validation.")

# 3. Handle missing values in the linked data
if is_trait_available:
    print("\nHandling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"After missing value handling, linked data shape: {linked_data.shape}")

# 4. Determine whether trait and demographic features are biased
if is_trait_available and not linked_data.empty and len(linked_data.columns) > 1:
    print("\nEvaluating feature bias...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Trait bias evaluation result: {is_biased}")
else:
    is_biased = False
    print("Skipping bias evaluation due to insufficient data.")

# 5. Final validation and save metadata
note = ""
if not is_trait_available:
    note = f"Dataset contains gene expression data but no {trait} measurements."
elif is_biased:
    note = f"Dataset contains {trait} data but its distribution is severely biased."

# Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Save the linked data if usable
print(f"\nDataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset is not usable for {trait} association studies. Data not saved.")